In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('german'))
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
import seaborn as sns

In [2]:
df = pd.read_csv('German.csv')

In [3]:
df.head()

,Sentence_id,Prior_Emotion,Sentence,Temporal_Distance,Intensity,Duration,Gender,City,Country,Worker_id,Time,Angst,Ekel,Freude,Scham,Schuld,Traurigkeit,Wut
0,370,Traurigkeit,"Ich fühlte ..., als mein Laptop kaputt ging un...",M,Mi,Epm,Ml,Dortmund,DEU,20,11/26/2018 19:00:21,0,0,0,0,0,0,5
1,88,Ekel,"Ich fühlte ..., als ich eine völlig verschmutz...",W,Mi,Epm,Ml,Ingolstadt,DEU,28,11/21/2018 18:15:56,0,5,0,0,0,0,0
2,76,Ekel,"Ich fühle ..., wenn ich in Hundescheiße getret...",M,S,Es,Ml,Wesel,DEU,5,11/21/2018 08:35:59,0,5,0,0,0,0,0
3,361,Traurigkeit,"Ich fühlte ..., als das Wetter sehr kalt und t...",T,Mi,Epm,Ml,Greven,DEU,20,11/24/2018 00:28:01,0,0,0,0,0,5,0
4,268,Schuld,"Ich fühlte ..., als ich einen Unfall mit dem A...",M,Mi,Ms,Ml,Krefeld,DEU,12,11/20/2018 19:31:08,0,0,0,0,5,0,0


In [4]:
df.drop(['Prior_Emotion', 'Temporal_Distance', 'Intensity', 'Duration', 'Gender', 'City', 'Country', 'Worker_id', 'Time'], axis = 1, inplace = True)

In [5]:
df_new = df.drop(['Sentence_id', 'Sentence'], axis=1)
counts = []
categories = list(df_new.columns.values)
for i in categories:
    counts.append((i, df[i].sum()))
df_stats = pd.DataFrame(counts, columns=['Emotion', 'no. sentences'])
df_stats

,Emotion,no. sentences
0,Angst,671
1,Ekel,663
2,Freude,806
3,Scham,625
4,Schuld,663
5,Traurigkeit,797
6,Wut,780


In [6]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(df['Sentence']).head()

,Sentence
0,"Ich fühlte ..., als mein Laptop kaputt ging und die Garantie schon abgelaufen war."
1,"Ich fühlte ..., als ich eine völlig verschmutzte Toillette betrat."
2,"Ich fühle ..., wenn ich in Hundescheiße getreten bin"
3,"Ich fühlte ..., als das Wetter sehr kalt und trüb wurde."
4,"Ich fühlte ..., als ich einen Unfall mit dem Auto meiner Freundin gebaut habe"


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\W', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text) 
    text = re.sub('\s+', ' ', text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.strip(' ')
    return text

In [8]:
df['Sentence'] = df['Sentence'].map(lambda com : clean_text(com))

In [9]:
df['Sentence'][0:10]

0    ich fühlte als mein laptop kaputt ging und die garantie schon abgelaufen war                                                  
1    ich fühlte als ich eine völlig verschmutzte toillette betrat                                                                  
2    ich fühle wenn ich in hundescheiße getreten bin                                                                               
3    ich fühlte als das wetter sehr kalt und trüb wurde                                                                            
4    ich fühlte als ich einen unfall mit dem auto meiner freundin gebaut habe                                                      
5    ich fühlte als mich mein exfreund betrogen hat                                                                                
6    ich fühlte auf meinen nachbarn weil er immer seine hecke nicht schneidet                                                      
7    ich fühlte als mein vater schwer erkrankte                             

In [10]:
df.iloc[: ,2:8]

,Angst,Ekel,Freude,Scham,Schuld,Traurigkeit
0,0,0,0,0,0,0
1,0,5,0,0,0,0
2,0,5,0,0,0,0
3,0,0,0,0,0,5
4,0,0,0,0,5,0
...,...,...,...,...,...,...
996,1,4,0,0,0,0
997,0,0,0,0,3,1
998,0,0,5,0,0,0
999,0,0,0,2,3,0


In [11]:
df.replace([2,3,4,5], 1, inplace = True)

In [12]:
df.iloc[: ,2:8]

,Angst,Ekel,Freude,Scham,Schuld,Traurigkeit
0,0,0,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,1,0
...,...,...,...,...,...,...
996,1,1,0,0,0,0
997,0,0,0,0,1,1
998,0,0,1,0,0,0
999,0,0,0,1,1,0


In [13]:
train, test = train_test_split(df, test_size=0.10, random_state=1, shuffle=True)

In [14]:
X_train = train.Sentence
X_test = test.Sentence
print(X_train.shape)
print(X_test.shape)

(900,)
(101,)


In [15]:
categories = ['Angst', 'Ekel', 'Freude', 'Scham', 'Schuld', 'Traurigkeit', 'Wut']

In [18]:
SVC_pipeline = Pipeline([
               ('tfidf', TfidfVectorizer(stop_words=stop_words, ngram_range = (1, 2), max_df = 0.85)),
               ('clf', CalibratedClassifierCV(OneVsRestClassifier(LinearSVC(), n_jobs=1))),
           ])

for category in categories:
    print('Emotion: {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print(' {}'.format(accuracy_score(test[category], prediction)))

Emotion: Angst
 0.8514851485148515
Emotion: Ekel
 0.9108910891089109
Emotion: Freude
 0.8514851485148515
Emotion: Scham
 0.7623762376237624
Emotion: Schuld
 0.7920792079207921
Emotion: Traurigkeit
 0.801980198019802
Emotion: Wut
 0.7326732673267327


In [17]:
#import pickle
#model = LinearSVC()
#filename = 'german_model.sav'
#pickle.dump(model, open(filename, 'wb'))

In [20]:
df_no_values = pd.read_csv('German_no_values.csv')
df_no_values['Sentence'] = df_no_values['Sentence'].map(lambda com : clean_text(com))
df_no_values.head()

,Sentence,Angst,Ekel,Freude,Scham,Schuld,Traurigkeit,Wut
0,ich fühlte als mein laptop kaputt ging und die garantie schon abgelaufen war,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ich fühlte als ich eine völlig verschmutzte toillette betrat,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ich fühle wenn ich in hundescheiße getreten bin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ich fühlte als das wetter sehr kalt und trüb wurde,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ich fühlte als ich einen unfall mit dem auto meiner freundin gebaut habe,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
X = df_no_values.Sentence

In [22]:
for category in categories:
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    y_proba = SVC_pipeline.predict_proba(X)[:,1]
    df_no_values[category] = y_proba

In [23]:
df_no_values.head()

,Sentence,Angst,Ekel,Freude,Scham,Schuld,Traurigkeit,Wut
0,ich fühlte als mein laptop kaputt ging und die garantie schon abgelaufen war,0.078012,0.016326,0.040261,0.094721,0.155112,0.108378,0.754558
1,ich fühlte als ich eine völlig verschmutzte toillette betrat,0.061692,0.839146,0.052194,0.143640,0.072898,0.111037,0.137684
2,ich fühle wenn ich in hundescheiße getreten bin,0.088389,0.999868,0.039617,0.075624,0.033768,0.084131,0.355536
3,ich fühlte als das wetter sehr kalt und trüb wurde,0.109723,0.017835,0.050810,0.100364,0.070125,0.793913,0.218308
4,ich fühlte als ich einen unfall mit dem auto meiner freundin gebaut habe,0.098170,0.008599,0.044593,0.117147,0.962995,0.072730,0.086774
